In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from PIL import Image
import random
from utils import (
    small_trigger_attack, watermark_trigger_attack, noised_trigger_attack,
    poison_dataset, poison_entire_testset
)

In [3]:
# config
with open("config.json", "r") as f:
    config = json.load(f)

alpha = config["alpha"]
x_coord_start = config["x_coord_start"]
y_coord_start = config["y_coord_start"]
gamma = config["gamma"]
poison_percentage = config["poison_percentage"]

In [4]:
# dataset cifar100
data_path = "./data/cifar100_data/"
train_data = datasets.CIFAR100(root=data_path, train=True, download=True)
test_data = datasets.CIFAR100(root=data_path, train=False, download=True)
train_images = [img for img, _ in train_data]
train_labels = [label for _, label in train_data]
test_images = [img for img, _ in test_data]
test_labels = [label for _, label in test_data]

In [5]:
# trigger
trigger = Image.open("trigger.jpg")
noised_trigger = Image.open("noised_trigger_epoch_100.png")


In [6]:
# poison methods
poison_attacks = {
    "small": small_trigger_attack,
    "watermark": watermark_trigger_attack,
    "noised": noised_trigger_attack
}

In [7]:
# training param
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 5
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [8]:
# loop through poison types
for poison_type, attack_fn in poison_attacks.items():
    print(f"\n**** Training ResNet18 with {poison_type} trigger")

    # poison data and test set
    trigger_img = noised_trigger if poison_type == "noised" else trigger

    # Customize parameters based on attack type
    attack_kwargs = {}
    if poison_type == "small":
        attack_kwargs = {
            "gamma": gamma,
            "x_coord_start": x_coord_start,
            "y_coord_start": y_coord_start
        }
    else:  # "watermark" or "noised"
        attack_kwargs = {
            "alpha": alpha
        }

    poisoned_train, poisoned_train_labels = poison_dataset(
        train_images, train_labels, attack_fn, trigger_img, poison_percentage,
        **attack_kwargs
    )

    poisoned_test, poisoned_test_labels = poison_entire_testset(
        test_images, test_labels, attack_fn, trigger_img,
        **attack_kwargs
    )

    # transform
    train_data_transformed = []
    for img, label in zip(poisoned_train, poisoned_train_labels):
        try:
            train_data_transformed.append((transform(img), label))
        except Exception as e:
            print(f"Error transforming training image: {e}")
            continue

    test_data_transformed = []
    for img, label in zip(poisoned_test, poisoned_test_labels):
        try:
            test_data_transformed.append((transform(img), label))
        except Exception as e:
            print(f"Error transforming test image: {e}")
            continue

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # model
    model = resnet18(weights="DEFAULT").to(device)
    model.fc = nn.Linear(model.fc.in_features, 100).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # train loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.5f}, Acc: {100.*correct/total:.2f}%")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Accuracy: {100.*correct/total:.2f}%")

    # save
    model_path = f"resnet18_cifar100_poison_{poison_type}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Saved model to {model_path}")

    print(f"Finished training on poison: {poison_type}")



**** Training ResNet18 with small trigger


KeyboardInterrupt: 